In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split


# reading the Treebank tagged sentences
nltk_data = list( nltk.corpus.treebank.tagged_sents( tagset ='universal') )
train_set , test_set = train_test_split( nltk_data , train_size =0.95 , test_size =0.05 , random_state =123)

# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
test_words_without_tags = [ tup [0] for sent in test_set for tup in sent ]

# ------------------------------------------------------------------------------------- #

training_words = [ word[0] for word in train_tagged_words ]
vocabulary = [ voc for voc in set(training_words) ]
print(f'The size of the vocabulary is: {len(vocabulary)}')

training_tags = [ word[1] for word in train_tagged_words ]
tags = [ lab for lab in set(training_tags)]
tags.append('OOV')      # out of vocabulary
print(f'The possible tags are: {tags}')

# initialize emission counts 
em_counts = np.zeros((len(vocabulary), len(tags)))
  
# iterating through the elements of list
for i in train_tagged_words:
  word = vocabulary.index(i[0])
  tag = tags.index(i[1])
  em_counts[word,tag] += 1  

# get probabilities 
em_prob = np.zeros((len(vocabulary)+1, len(tags)))

for row in range(0, len(em_counts)):
  rows = em_counts[row, :]
  em_prob[row, :] = [val/sum(rows) for val in rows]

em_prob[-1, :] = [1/len(tags) for i in range(len(tags))]

# create matrix
em_matrix = np.zeros((len(vocabulary), len(tags)))
i=0

for col in range(0,12):
  column = em_counts[:, col]
  probs = [ val/sum(column) for val in column]
  em_matrix[:, col] = probs

# initialize transition counts
trans_counts = np.zeros((len(tags), len(tags)))
tag2_counts = 12 * [0]

# iterating through the elements of list
for i in range(0,len(train_tagged_words)-1):
  curr_tag = tags.index(train_tagged_words[i][1])
  next_tag = tags.index(train_tagged_words[i+1][1])
  trans_counts[curr_tag, next_tag] += 1
  tag2_counts[next_tag] += 1

# calculated above as dictionary, calc in matrix
trans_matrix = np.zeros((len(tags), len(tags)))
i=0

for col in range(0,12):
  column = trans_counts[:, col]
  probs = [ val/sum(column) for val in column]
  trans_matrix[:, col] = probs

print('DONE')

The size of the vocabulary is: 12102
The possible tags are: ['NUM', '.', 'CONJ', 'PRT', 'VERB', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADV', 'X', 'PRON', 'OOV']
DONE


variables:

z_ij  e {0,1}   binary        label for index i is j

z_ijk e {0,1}                 label for index i-1 is j and index i is k



objective:

sum_i ( sum_j (log(P(x_i | y_j) * z_ij) + sum_jk (log(P(y_i=k|y_i-1=j) * z_ijk)))

                   emission                           transition

linear in z_ij and z_ijk


constraints:

sum_j (z_ij) = 1              for all i

z_ijk =< z_ij                 for all i,j,k

z_ijk =< z_i-1_j + z_ik -1    for all i,j,k

ILP Solver: https://realpython.com/linear-programming-python/#linear-programming-python-implementation
https://medium.com/opex-analytics/optimization-modeling-in-python-pulp-gurobi-and-cplex-83a62129807a
https://stackoverflow.com/questions/57025856/gurobi-constraints-and-objective-function

In [6]:
import math
import gurobipy as grb

emission = np.zeros((len(test_words_without_tags), len(tags)))
for row in range(len(test_words_without_tags)):
  w = test_words_without_tags[row]
  if w in vocabulary:
    emission[row, :] = em_matrix[vocabulary.index(w),:]
  else:
    emission[row, :] = [1/len(tags) for _ in range(len(tags))]

transition = trans_matrix
n = len(test_words_without_tags)  # upperbound of sigma sum_i
m = len(tags)                     # upperbound of sigma sum_j
set_I  = range(1, n+1)
set_J  = range(1, m+1)


m = grb.Model(name="ILP Model")

# since z_ij is Binary
z = m.addVar(len(test_words_without_tags),  len(tags), vtype=grb.GRB.BINARY)
m.update()

# IMPORTANT: z_i_jk = z_i-1_j | z_i_k = ( z[i-1, j] | z[i, k] ) = z[i-1, j] * z[i, k] 

# CONSTRAINTS
# <= constraints  : z_ijk =< z_ij                 for all i,j,k
# constraints = {i,j,k : 
# m.addConstr(
#         lhs=z[i,k]] * z[i-1,j],
#         sense=grb.GRB.LESS_EQUAL,
#         rhs= z[i,j], 
#         name="constraint_{0}".format(j))
#     for j in set_J for i in set_I for k in set_k}

# >= constraints  : z_ijk => z_i-1_j + z_ik -1    for all i,j,k
# for k in set_J:
#   for j in set_J:
#     for i in set_I:
#       # m.addConstr(( z[i-1, j]*z[i, k] ) >= z[i])
#       m.addConstr(( z[i, k] ) >= z[i])

# == constraints  : sum_j z_ij = 1     for all i
# for k in set_J:
# for i in set_I:
#   m.addConstr( grb.quicksum(z[i,j] for j in set_J) == 1 )

m.addConstrs( ( grb.quicksum(z[i,j] for j in set_J) == 1 ) for i in set_I )



# objective       : sum_i ( sum_j (log(emission[i,j]) * z_ij) + sum_jk (log(transition[k,j]) * z_ijk)))
sum_j = grb.quicksum(math.log(emission[i,j]) * z[i,j] for j in set_J)
# sum_jk = grb.quicksum(grb.quicksum(math.log(transition[i,j]) * (z[i-1, j] | z[i, k])) for j in set_J for k in set_J )
sum_jk = grb.quicksum(grb.quicksum(math.log(transition[i,j]) * z[i, k] * (z[i-1, j])) for j in set_J for k in set_J )
objective = grb.quicksum((sum_j + sum_jk) for i in set_I)

# optimize
m.ModelSense = grb.GRB.MAXIMIZE
m.setObjective(objective)
m.optimize()

import pandas as pd
opt_df = pd.DataFrame.from_dict(z, orient="index", 
                                columns = ["variable_object"])
opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, names=["column_i", "column_j"])
opt_df.reset_index(inplace=True)
opt_df["solution_value"] = opt_df["variable_object"]

TypeError: 'Var' object is not subscriptable